<a href="https://colab.research.google.com/github/kamehl/ex21/blob/main/SECCON_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SECCON 2022

##crypoto

###pqpq

In [ ]:
!pip install pycryptodome

In [8]:
e = 131074
n = 587926815910957928506680558951380405698765957736660571041732511939308424899531125274073420353104933723578377320050609109973567093301465914201779673281463229043539776071848986139657349676692718889679333084650490543298408820393827884588301690661795023628407437321580294262453190086595632660415087049509707898690300735866307908684649384093580089579066927072306239235691848372795522705863097316041992762430583002647242874432616919707048872023450089003861892443175057
c1 = 92883677608593259107779614675340187389627152895287502713709168556367680044547229499881430201334665342299031232736527233576918819872441595012586353493994687554993850861284698771856524058389658082754805340430113793873484033099148690745409478343585721548477862484321261504696340989152768048722100452380071775092776100545951118812510485258151625980480449364841902275382168289834835592610827304151460005023283820809211181376463308232832041617730995269229706500778999
c2 = 46236476834113109832988500718245623668321130659753618396968458085371710919173095425312826538494027621684566936459628333712619089451210986870323342712049966508077935506288610960911880157875515961210931283604254773154117519276154872411593688579702575956948337592659599321668773003355325067112181265438366718228446448254354388848428310614023369655106639341893255469632846938342940907002778575355566044700049191772800859575284398246115317686284789740336401764665472
cm = 357982930129036534232652210898740711702843117900101310390536835935714799577440705618646343456679847613022604725158389766496649223820165598357113877892553200702943562674928769780834623569501835458020870291541041964954580145140283927441757571859062193670500697241155641475887438532923910772758985332976303801843564388289302751743334888885607686066607804176327367188812325636165858751339661015759861175537925741744142766298156196248822715533235458083173713289585866

In [ ]:
from Crypto.Util.number import *

eが偶数なので次のように展開できる。

>$c2 = (p-q)^{e} = p^{e} +\displaystyle \sum_{i,j}^{0<i,j<e}x_{ij}p^iq^j+q^{e}$

※ポイントは$q^{e}$の前が「＋」になる部分。

$c1$と$c2$を足したり引いたりすると$p$と$q$の倍数がそれぞれ作れるので、
$n$とユークリッドの互助法を使えば$(p,q,r)$がすべて求まる。

※$p$か$q$の倍数を作ればOKだけど、$pq$の倍数になってもダメ。

In [ ]:
p = GCD(n, c1+c2)
q = GCD(n, c1-c2)
r = n//q//p
print(f"p = {p}")
print(f"q = {q}")
print(f"r = {r}")

p = 7572427786695057270624844967644562609112132599800420296747189080920032359205995588384031542287784540006438555802994008688795974493684400576592403320929717
q = 8609258896430210586523688955272794335561428099377427081622836355194006054569349679983850344916908011330202034512905353365631416251631307084038768336538857
r = 9018251874561850467651399512661829039310834429345808807288228370045576292997274498659156953954383290793552486677903139680704353709352146165598701061994853


eが２の倍数なので$\phi(n) = (p-1)(q-1)(r-1)$での逆元が計算できない。

でも、$e/2$は素数だから$e/2$の逆元$d$は計算できる。

ちなみに

>$ed\ mod\ \phi(n) = 2$

なので

>$cm^{d} = (m^{e})^{d} = m^{ed} = m^{k\phi(n)+2} = m^{2}\ mod\ n$

ただし、$k$は任意の整数。あと、

>$cm^{d} = m_{p}^{2}\ mod\ p$<br>
>$cm^{d} = m_{q}^{2}\ mod\ q$<br>
>$cm^{d} = m_{r}^{2}\ mod\ r$

も計算可能。

In [ ]:
phi = (p-1) * (q-1) * (r-1)
d = inverse(e//2, phi)

m2_mod_p = pow(cm, d, p)
m2_mod_q = pow(cm, d, q)
m2_mod_r = pow(cm, d, r)


法が奇素数である平方余剰は簡単に計算できるらしいので、下のURLのソースをパクって対応。

https://rosettacode.org/wiki/Tonelli-Shanks_algorithm#Python

In [ ]:
def legendre(a, p):
    return pow(a, (p - 1) // 2, p)

def tonelli(n, p):
    assert legendre(n, p) == 1, "not a square (mod p)"
    q = p - 1
    s = 0
    while q % 2 == 0:
        q //= 2
        s += 1
    if s == 1:
        return pow(n, (p + 1) // 4, p)
    for z in range(2, p):
        if p - 1 == legendre(z, p):
            break
    c = pow(z, q, p)
    r = pow(n, (q + 1) // 2, p)
    t = pow(n, q, p)
    m = s
    t2 = 0
    while (t - 1) % p != 0:
        t2 = (t * t) % p
        for i in range(1, m):
            if (t2 - 1) % p == 0:
                break
            t2 = (t2 * t2) % p
        b = pow(c, 1 << (m - i - 1), p)
        r = (r * b) % p
        c = (b * b) % p
        t = (t * c) % p
        m = i
    return r


ただし、平方余剰は２個存在する。平方根が２個存在するのと同じ感じ。

In [ ]:
mp1 = tonelli(m2_mod_p, p)
mp2 = p - mp1

mq1 = tonelli(m2_mod_q, q)
mq2 = q - mq1

mr1 = tonelli(m2_mod_r, r)
mr2 = r - mr1

あとは中国人の余剰定理で$m$の候補をしらみつぶしに探す。

※平方余剰は2個あるので、８つのパターンをすべて考える必要がある。

In [ ]:
!pip install sympy

In [ ]:
import sympy.ntheory.modular

In [ ]:
X = [p,q,r]
for i in (mp1, mp2):
    for j in (mq1, mq2):
        for k in (mr1, mr2):
            Y = [i, j, k]
            (x, y) = sympy.ntheory.modular.crt(X,Y)
            flag = long_to_bytes(x)
            if b'SECCON' in flag:
                print(flag)


b'\x12=\x04q\x9b\xb8\x1c\x10C\x02\x1e\x13`\xac>A\x9c\xf9\x9d\xc2\x83\xc2\xcd\x15\x97\x86\x8e\xd2\x85*s\r\x18~\x9b\xbai\xb1\x07\xacF\x0f\xfcrZ\xf1\xd0\x1f\xb0q\xe4\xbf\xd2\x87G\x1b\xdc\xd2u\x97\xb3\xcc?\xba\xba@\xae\x96\xdc\x1b\x10\xd3\x00f\nH\x99d\xf7{\xea \x82T\xf5\x03\x81\xd0:\r\x8d\xa6P\x92\xa0\x1d\x91n u6}:\x98\r\xa0\xbc\xe5\x84y\x01\x89\xa4P\xf4\xf9\xe4\xf2\x95\x8d\x85\x11\xfezN\x06- e(\x80\xd2\x01\x8e\x94&\xf7amQ\x08@\xd4w\x8e\xbbP\xfa\x17SECCON{being_able_to_s0lve_this_1s_great!}'


以下、サンプルコード。

In [5]:
from Crypto.Util.number import *
import sympy.ntheory.modular

#ネットからパクったコード
#https://rosettacode.org/wiki/Tonelli-Shanks_algorithm#Python
#
def legendre(a, p):
    return pow(a, (p - 1) // 2, p)

def tonelli(n, p):
    assert legendre(n, p) == 1, "not a square (mod p)"
    q = p - 1
    s = 0
    while q % 2 == 0:
        q //= 2
        s += 1
    if s == 1:
        return pow(n, (p + 1) // 4, p)
    for z in range(2, p):
        if p - 1 == legendre(z, p):
            break
    c = pow(z, q, p)
    r = pow(n, (q + 1) // 2, p)
    t = pow(n, q, p)
    m = s
    t2 = 0
    while (t - 1) % p != 0:
        t2 = (t * t) % p
        for i in range(1, m):
            if (t2 - 1) % p == 0:
                break
            t2 = (t2 * t2) % p
        b = pow(c, 1 << (m - i - 1), p)
        r = (r * b) % p
        c = (b * b) % p
        t = (t * c) % p
        m = i
    return r

#output.txtのデータ
#
e = 131074
n = 587926815910957928506680558951380405698765957736660571041732511939308424899531125274073420353104933723578377320050609109973567093301465914201779673281463229043539776071848986139657349676692718889679333084650490543298408820393827884588301690661795023628407437321580294262453190086595632660415087049509707898690300735866307908684649384093580089579066927072306239235691848372795522705863097316041992762430583002647242874432616919707048872023450089003861892443175057
c1 = 92883677608593259107779614675340187389627152895287502713709168556367680044547229499881430201334665342299031232736527233576918819872441595012586353493994687554993850861284698771856524058389658082754805340430113793873484033099148690745409478343585721548477862484321261504696340989152768048722100452380071775092776100545951118812510485258151625980480449364841902275382168289834835592610827304151460005023283820809211181376463308232832041617730995269229706500778999
c2 = 46236476834113109832988500718245623668321130659753618396968458085371710919173095425312826538494027621684566936459628333712619089451210986870323342712049966508077935506288610960911880157875515961210931283604254773154117519276154872411593688579702575956948337592659599321668773003355325067112181265438366718228446448254354388848428310614023369655106639341893255469632846938342940907002778575355566044700049191772800859575284398246115317686284789740336401764665472
cm = 357982930129036534232652210898740711702843117900101310390536835935714799577440705618646343456679847613022604725158389766496649223820165598357113877892553200702943562674928769780834623569501835458020870291541041964954580145140283927441757571859062193670500697241155641475887438532923910772758985332976303801843564388289302751743334888885607686066607804176327367188812325636165858751339661015759861175537925741744142766298156196248822715533235458083173713289585866

#nの因数分解
#GCDはpycryptodomeを利用
#
p = GCD(n, c1+c2)
q = GCD(n, c1-c2)
r = n//q//p

#eが偶数なので逆元が求まらないので、e//2の逆元を計算
#ただし、このdで復号すると平文の２乗しか計算できない
#
phi = (p-1) * (q-1) * (r-1)
d = inverse(e//2, phi)

#奇素数を法とする平方剰余は簡単に計算できるらしいのでプログラムごとパクる
#ただし、答えは２個出てくる（平方根が＋と−で二つあるのと同じ）
#https://ja.wikipedia.org/wiki/平方剰余
#
m2_mod_p = pow(cm, d, p)
m2_mod_q = pow(cm, d, q)
m2_mod_r = pow(cm, d, r)

mp1 = tonelli(m2_mod_p, p)
mp2 = p - mp1

mq1 = tonelli(m2_mod_q, q)
mq2 = q - mq1

mr1 = tonelli(m2_mod_r, r)
mr2 = r - mr1

#平文のp,q,rの剰余が分かったのであとは中国人の剰余定理を使えばflagゲット
#ただし、剰余はp,q,rのそれぞれで２個ずつあるので、フラグっぽいのをしらみつぶしで探す
#あと、crtはsympyのやつを利用
#
X = [p,q,r]
for i in (mp1, mp2):
    for j in (mq1, mq2):
        for k in (mr1, mr2):
            Y = [i, j, k]
            (x, y) = sympy.ntheory.modular.crt(X,Y)
            flag = long_to_bytes(x)
            if b'SECCON' in flag:
                print(flag)

b'\x12=\x04q\x9b\xb8\x1c\x10C\x02\x1e\x13`\xac>A\x9c\xf9\x9d\xc2\x83\xc2\xcd\x15\x97\x86\x8e\xd2\x85*s\r\x18~\x9b\xbai\xb1\x07\xacF\x0f\xfcrZ\xf1\xd0\x1f\xb0q\xe4\xbf\xd2\x87G\x1b\xdc\xd2u\x97\xb3\xcc?\xba\xba@\xae\x96\xdc\x1b\x10\xd3\x00f\nH\x99d\xf7{\xea \x82T\xf5\x03\x81\xd0:\r\x8d\xa6P\x92\xa0\x1d\x91n u6}:\x98\r\xa0\xbc\xe5\x84y\x01\x89\xa4P\xf4\xf9\xe4\xf2\x95\x8d\x85\x11\xfezN\x06- e(\x80\xd2\x01\x8e\x94&\xf7amQ\x08@\xd4w\x8e\xbbP\xfa\x17SECCON{being_able_to_s0lve_this_1s_great!}'
